In [3]:
import bs4
import pandas as pd
from bs4 import BeautifulSoup
from requests import get
import numpy as np

In [4]:
def get_data():
    title = []
    price = []
    pages = ["","?s=120","?s=240","?s=360","?s=480","?s=600","?s=720","?s=840","?s=960","?s=1080"]

    ## iterate through the pages to collect desired dataset
    for x in pages:
        url = 'https://newyork.craigslist.org/search/bka' + x
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        type(html_soup)
        books = html_soup.find_all('li', class_ = 'result-row')
    
        ## extract all the data from books
        for every_book in books:
            ## collect title of the book:
            name = every_book.p.a.text
            title.append(name)
            ## collect price of the book:
            prices = every_book.p.find('span',class_='result-price').text
            price.append(prices)
            test_df = pd.DataFrame({'title': title,
                                    'price':price})
            
    # take the first 1000 rows since data exceeds 1000 rows
    data = test_df[0:1000]
    return data

In [7]:
Data=get_data()

In [8]:
len(Data)

1000

In [9]:
Data.loc[0:5]

,title,price
0,Modern Refrigeration and Air Conditioning,$45
1,CDL MOTORCYCL DRIVING PERMIT EXAM GURANTEE PAS...,$25
2,Coffee Table Book,$5
3,"Free Kumon Sheets Math level G, H with Kumon e...",$0
4,mickey mantle @ roger maris books,$15
5,Bat Boy book,$5


In [10]:
Data.loc[0:5,"title"]

0            Modern Refrigeration and Air Conditioning
1    CDL MOTORCYCL DRIVING PERMIT EXAM GURANTEE PAS...
2                                    Coffee Table Book
3    Free Kumon Sheets Math level G, H with Kumon e...
4                mickey mantle @ roger maris     books
5                                         Bat Boy book
Name: title, dtype: object

In [21]:
## search books by title (case sensitive)
def search_by_title():
    TEST = input("Search name of the title: ")
    title = np.column_stack([Data[col].str.contains(TEST) for col in ['title']])
    return print(Data.loc[title.any(axis=1)])

In [23]:
def search_by_title():
    TEST = input("Search name of the title: ")
    title = np.column_stack([Data[col].str.contains(TEST.lower()) for col in ['title']])
    return print(Data.loc[title.any(axis=1)])

In [25]:
search_by_title()

Search name of the title: Modern
Empty DataFrame
Columns: [title, price]
Index: []


In [ ]:
TEST = input("Search name of the title: ")
title = Data.loc[Data['title']= str.contains(TEST)]
print(title)

In [17]:
search_by_title()

Search name of the title: mm
                                                 title price
80    English Composition and Grammar: Complete Course  $100
219  The Mammoth Book of Historical and More Whodun...    $9
328  Effective Organisational Communication: Perspe...   $30
334  Modernist Communities across Cultures and Medi...   $50
422                      Programming and finance books   $40
636  Assorted Great Programming/Web Development Boo...    $5
659             Computer and Programming Related Books    $0
735                     Computer Dummie Books for Sale    $0
746  Mens european fashion magazine library for sal...  $360
752   L'uomo Vogue Homme mens fashion magazine Library  $480
763  Robert's Rules for Dummies - Parliamentary Pro...    $5
822   English Composition and Grammar: Complete Course  $100
934   English Composition and Grammar: Complete Course  $100


In [203]:
## search books by price (with $ sign)
def search_by_price():
    TEST = input("Search price of the book: ")
    price = Data.loc[Data['price']==TEST]
    return price

In [204]:
print(search_by_price())

Search price of the book: $4
                                                 title price
28               New-adventure romance novel On A Whim    $4
63                        100 Things to Ponder (ebook)    $4
216                   Quick and Easy Investing E-Guide    $4
217                   Quick and Easy Investing E-Guide    $4
219                   Quick and Easy Investing E-Guide    $4
270                    GHOST STORIES of NEW YORK STATE    $4
302                         Art History Books for Sale    $4
474                       The Tibetan Book of the Dead    $4
563  John Kovalic's Dork Tower: Living La Vida Dork...    $4
761                            Broadway Show Playbills    $4


In [15]:
## pagination
from tempfile import NamedTemporaryFile
import webbrowser
from sklearn.datasets import load_iris


def df_html(df):
    """HTML table with pagination and other goodies"""
    df_html = df.to_html()
    return base_html % df_html

In [16]:
def df_window(df):
    """Open dataframe in browser window using a temporary file"""
    with NamedTemporaryFile(delete=False, suffix='.html') as f:
        f.write(df_html(df))
    webbrowser.open(f.name)

In [22]:
df = pd.DataFrame(Data, columns=Data.feature_names)

df_window(df)

AttributeError: 'DataFrame' object has no attribute 'feature_names'

In [ ]:
## pagination
import dash
from dash.dependencies import Input, Output
import dash_table
import dash_html_components as html
import dash_core_components as dcc



Data[' index'] = range(1, len(Data) + 1)
app = dash.Dash(__name__)
PAGE_SIZE = 15

app.layout = html.Div([
    html.Div('Data table pagination test'),
    dcc.Dropdown(
        id='select_page_size',
        options=[{'label': '5', 'value': 5}, {'label': '10', 'value': 10}, {'label': '15', 'value': 15}],
        value=5
    ),
    html.Div(dash_table.DataTable(
        id='datatable-paging',
        columns=[{"name": i, "id": i} for i in Data.columns],
        data=Data.to_dict("rows"),
        pagination_settings={
            'current_page': 0,
            'page_size': PAGE_SIZE
        },
    ))
])

@app.callback(
    Output('datatable-paging', 'pagination_settings'),
    [Input('select_page_size', 'value')])
def update_graph(page_size):
    return {
        'current_page': 0,
        'page_size': page_size
    }

if __name__ == '__main__':
    app.run_server(debug=True)